In [1]:
import numpy as np

Координаты пикселя.

|         | **m=1**  | **m=2**  |
|---------|----------|----------|
| **n=2** | m=1, n=2 | m=2, n=2 |
| **n=1** | m=1, n=1 | m=2, n=1 |

In [2]:
m = 2 # > 0
n = 1 # > 0
tensor_type = 'WW' # 'VV', 'VW', 'WW', 'WV'
lvl = 2

Создаем масштабирующие вектора.

In [3]:
N = 8
K = 2
V = np.empty(K, dtype='object')
W = np.empty(K, dtype='object')
for k in range(K):
    V[k] = np.zeros((int(N/2**(k+1)),N))
    W[k] = np.zeros((int(N/2**(k+1)),N))


V[0][0] = [1/np.sqrt(2), 1/np.sqrt(2), 0, 0, 0, 0, 0, 0]
V[0][1] = [0, 0, 1/np.sqrt(2), 1/np.sqrt(2), 0, 0, 0, 0]
V[0][2] = [0, 0, 0, 0, 1/np.sqrt(2), 1/np.sqrt(2), 0, 0]
V[0][3] = [0, 0, 0, 0, 0, 0, 1/np.sqrt(2), 1/np.sqrt(2)]

W[0][0] = [1/np.sqrt(2), -1/np.sqrt(2), 0, 0, 0, 0, 0, 0]
W[0][1] = [0, 0, 1/np.sqrt(2), -1/np.sqrt(2), 0, 0, 0, 0]
W[0][2] = [0, 0, 0, 0, 1/np.sqrt(2), -1/np.sqrt(2), 0, 0]
W[0][3] = [0, 0, 0, 0, 0, 0, 1/np.sqrt(2), -1/np.sqrt(2)]

V[1][0] = [1/2, 1/2, 1/2, 1/2, 0, 0, 0, 0]
V[1][1] = [0, 0, 0, 0, 1/2, 1/2, 1/2, 1/2]

W[1][0] = [1/2, 1/2, -1/2, -1/2, 0, 0, 0, 0]
W[1][1] = [0, 0, 0, 0, 1/2, 1/2, -1/2, -1/2]

for k in range(K):
    V[k] = np.matrix(V[k])
    W[k] = np.matrix(W[k])

# Тензорное произведение

Матрицы преобразования 1-го уровня для N и N/2:

In [4]:
Mdir = np.empty(K, dtype='object')
for k in range(K):
    Mdir[k] = np.concatenate((V[0][:(N//2**(k+1)),:(N//2**k)], W[0][:(N//2**(k+1)),:(N//2**k)]), axis = 0)

In [5]:
if tensor_type == 'WW':
    tensor_mtrx = np.transpose(np.flip(W[lvl-1][n-1])) * W[lvl-1][m-1]
elif tensor_type == 'VV':
    tensor_mtrx = np.transpose(np.flip(V[lvl-1][n-1])) * V[lvl-1][m-1]
elif tensor_type == 'VW':
    tensor_mtrx = np.transpose(np.flip(W[lvl-1][n-1])) * V[lvl-1][m-1]
elif tensor_type == 'WV':
    tensor_mtrx = np.transpose(np.flip(V[lvl-1][n-1])) * W[lvl-1][m-1]
else:
    print('Error: tensor_type is not correct')

Результирующая субматрица располагается в углу, координаты которого задаются параметрами **m**, **n**.

Параметр tensor_type ('VV', 'VW', 'WW', 'WV') определяет наполнение этой субматрицы.

In [6]:
tensor_mtrx

matrix([[ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
        [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
        [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
        [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
        [ 0.  ,  0.  ,  0.  ,  0.  , -0.25, -0.25,  0.25,  0.25],
        [ 0.  ,  0.  ,  0.  ,  0.  , -0.25, -0.25,  0.25,  0.25],
        [ 0.  ,  0.  ,  0.  ,  0.  ,  0.25,  0.25, -0.25, -0.25],
        [ 0.  ,  0.  ,  0.  ,  0.  ,  0.25,  0.25, -0.25, -0.25]])

Вариант с рандомной матрицей для обобщенного тестирования этого блокнота:

In [7]:
'''
np.random.seed(42)
tensor_mtrx = np.random.randint(-9, 9, size=(N, N))
print(np.round(tensor_mtrx,1))
'''

'\nnp.random.seed(42)\ntensor_mtrx = np.random.randint(-9, 9, size=(N, N))\nprint(np.round(tensor_mtrx,1))\n'

# Прямое преобразование

## Преобразование 1-го уровня

In [8]:
tr = np.empty(K, dtype='object')

Преобразование выполняется в следующей последовательности:

1. Преобразование строк.
2. Изменение порядка строк (bottom-up).
3. Преобразование столбцов.
4. Изменение порядка строк (bottom-up).

In [9]:
tr[0] = np.flipud(Mdir[0] * np.flipud(tensor_mtrx * np.transpose(Mdir[0])))

В результате в левом нижнем углу формируется субматрица размера N/2xN/2. В этой субматрице ненулевыми являются только 4 значения (массив 2x2), расположение которого в рамках субматрицы определяется только параметрами **m**, **n**.

In [10]:
print(np.round(tr[0],1))

[[ 0.   0.   0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.  -0.   0.   0.   0.   0. ]
 [ 0.   0.  -0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0.   0.   0.   0. ]
 [ 0.   0.  -0.5  0.5  0.   0.   0.   0. ]
 [ 0.   0.   0.5 -0.5  0.   0.   0.   0. ]]


## Преобразование 2-го уровня

Преобразуется исключительно левый нижний угол матрицы преобразования 1-го уровня.

In [11]:
tr[1] = np.flipud(Mdir[1] * np.flipud(tr[0][N//2:,:N//2] * np.transpose(Mdir[1])))
print(np.round(tr[1],1))

[[0. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


В результате формируется матрица размера N/2xN/2.

Только один элемент матрицы принимает ненулевой значение. Координаты этого элемента определяются следующими параметрами:

1. Параметр **tensor_type** локализует "угол" матрицы.

| VW | WW |

| VV | WV |

2. Параметры **m**, **n** определяют координаты в рамках "угла".

Встраиваем эту субматрицу размера N/2xN/2 в левый нижний угол итоговой матрицы размера NxN.

In [12]:
pixel_mtrx = tr[0].copy()
pixel_mtrx[N//2:,:N//2] = tr[1]
print(np.round(pixel_mtrx,1))

[[ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. -0.  0.  0.  0.  0.]
 [ 0.  0. -0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]]


# Преобразование с помощью PyWavelets.

Повторно выполним аналогичное преобразование с помощью библиотечных функций.

In [13]:
import pywt
from Functions.combine_matrices import combine_matrices

## Преобразование 1-го уровня

In [14]:
wavelet = 'haar'
pywt_tr = np.empty(K, dtype='object')

cA, (cH, cV, cD) = pywt.dwt2(tensor_mtrx, wavelet)
pywt_tr[0] = combine_matrices(cA, -cH, cV, -cD)

print(np.round(pywt_tr[0],1))

[[-0.  -0.  -0.  -0.  -0.  -0.  -0.  -0. ]
 [-0.  -0.  -0.  -0.  -0.  -0.  -0.  -0. ]
 [-0.  -0.  -0.  -0.  -0.  -0.  -0.  -0. ]
 [-0.  -0.  -0.  -0.  -0.  -0.  -0.  -0. ]
 [ 0.   0.   0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0.   0.   0.   0. ]
 [ 0.   0.  -0.5  0.5  0.   0.   0.   0. ]
 [ 0.   0.   0.5 -0.5  0.   0.   0.   0. ]]


## Преобразование 2-го уровня

In [15]:
cA, (cH, cV, cD) = pywt.dwt2(pywt_tr[0][N//2:,:N//2], wavelet)
pywt_tr[1] = combine_matrices(cA, -cH, cV, -cD)
print(np.round(pywt_tr[1],1))

[[-0. -0. -0. -0.]
 [-0. -0. -0.  1.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]


Теперь в составе полной матрицы NxN

In [16]:
pywt_pixel_mtrx = pywt_tr[0].copy()
pywt_pixel_mtrx[N//2:,:N//2] = pywt_tr[1]
print(np.round(pywt_pixel_mtrx,1))

[[-0. -0. -0. -0. -0. -0. -0. -0.]
 [-0. -0. -0. -0. -0. -0. -0. -0.]
 [-0. -0. -0. -0. -0. -0. -0. -0.]
 [-0. -0. -0. -0. -0. -0. -0. -0.]
 [-0. -0. -0. -0.  0.  0.  0.  0.]
 [-0. -0. -0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]]


# Обратное преобразование

In [17]:
from Functions.split_matrices import split_matrices
pywt_inv_tr = np.empty(K, dtype='object')

## Выполняем обратное преобразование 1й раз

In [18]:
cA, (cH, cV, cD) = split_matrices(pywt_pixel_mtrx[N//2:,:N//2])
pywt_inv_tr[1] = pywt_pixel_mtrx.copy()
pywt_inv_tr[1][N//2:,:N//2] = pywt.idwt2((cA, (-cH, cV, -cD)), wavelet)
print(np.round(pywt_inv_tr[1],1))

[[-0.  -0.  -0.  -0.  -0.  -0.  -0.  -0. ]
 [-0.  -0.  -0.  -0.  -0.  -0.  -0.  -0. ]
 [-0.  -0.  -0.  -0.  -0.  -0.  -0.  -0. ]
 [-0.  -0.  -0.  -0.  -0.  -0.  -0.  -0. ]
 [ 0.   0.   0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0.   0.   0.   0. ]
 [ 0.   0.  -0.5  0.5  0.   0.   0.   0. ]
 [ 0.   0.   0.5 -0.5  0.   0.   0.   0. ]]


## Выполняем обратное преобразование 2й раз
В результате получаем исходный тензор

In [19]:
cA, (cH, cV, cD) = split_matrices(pywt_inv_tr[1])
pywt_inv_tr[0] = pywt.idwt2((cA, (-cH, cV, -cD)), wavelet)
print(np.round(pywt_inv_tr[0],2))

[[ 0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.   -0.25 -0.25  0.25  0.25]
 [ 0.    0.    0.    0.   -0.25 -0.25  0.25  0.25]
 [ 0.    0.    0.    0.    0.25  0.25 -0.25 -0.25]
 [ 0.    0.    0.    0.    0.25  0.25 -0.25 -0.25]]
